Scrape the Python ITJobsWatch page. Showcase:
1. Data Ingestion
2. Data Wrangling
3. Data Analysis
4. Data Visualisation

In [114]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

x = requests.get('https://www.itjobswatch.co.uk/jobs/uk/python.do')
soup = BeautifulSoup(x.text)

Parse a single table into pandas

In [115]:
def get_info_from_row(row):
    return [tag.get_text() for tag in row.find_all('tr') if len(tag.get_text()) < 50]

In [116]:
def get_info_from_section(section):
    return [get_info_from_row(row) for row in section.find_all('td') if len(get_info_from_row(row)) != 0][:14]

In [117]:
def get_job_stats_df(soup):
    info = get_info_from_section(soup.find_all('section')[0])

    dict = {'2024' : [row[1] for row in info],
            '2023' : [row[2] for row in info],
            '2022' : [row[3] for row in info]}

    return pd.DataFrame(dict, index=[row[0] for row in info])

In [132]:
def get_skill_stats_df(soup):
    related_skill_texts = [text.split("%") for table in get_info_from_section(soup.find(attrs={'id':'related_skills'})) for text in table]

    dict = {'Percentage' : [skill_text[0].split("(")[1] for skill_text in related_skill_texts],
            'Skill' : [skill_text[1][1:].strip() for skill_text in related_skill_texts]}

    return pd.DataFrame(dict)

In [133]:
print(get_skill_stats_df(soup))

   Percentage                    Skill
0       38.57                      AWS
1       33.40                      SQL
2       32.02                    Agile
3       30.93     Software Engineering
4       28.88                    Azure
5       28.06                  Finance
6       27.35                     Java
7       25.72                   DevOps
8       22.29          Problem-Solving
9       22.14                    CI/CD
10      20.62                   Degree
11      20.26            Social Skills
12      20.04                    Linux
13      19.24               JavaScript
14      17.03               Kubernetes
15      16.65         Machine Learning
16      16.35                   Docker
17      15.34                Analytics
18      14.98                      GCP
19      14.70         Security Cleared
20      14.53                     Bash
21      13.19                       C#
22      13.16  Artificial Intelligence
23      13.00         Computer Science
24      12.97            

In [121]:
def get_category_stats_df(soup, i, name):
    info = get_info_from_section(soup.find_all('table')[6].find_all('table')[i])
    
    dict =  {
            'Category' : name,
            'Ranking' : [row[0] for row in info],
            'Percentage' : [row[1] for row in info],
            'Skill' : [row[2] for row in info]
            }

    return pd.DataFrame(dict)

In [122]:
df_genr = get_category_stats_df(soup, 7, "General")
df_jobs = get_category_stats_df(soup, 8, "Job")
df_libs = get_category_stats_df(soup, 9, "Library")
df_lang = get_category_stats_df(soup, 13, "Language")

df = pd.concat([df_genr, df_jobs, df_libs, df_lang])

Create df containing the following features for a skill:
- Name of skill
- Rank change
- % of all permanent jobs
- Category
- % of category
- Median annual salary
- Median annual salary (excl London)


Scrape results page to get Skill pages in descending order
- Build df of Skill features
- Build association table